###Importing libraries

In [ ]:
!pip install beautifulsoup4
!pip install requests

In [1]:
import numpy as np
import pandas as pd

###Importing dataset (from google drive)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/1_LiveProjects/Project2_Sentiment_Analysis
!ls

/content/drive/MyDrive/1_LiveProjects/Project2_Sentiment_Analysis
a1_RestaurantReviews_HistoricDump.tsv  c1_BoW_Sentiment_Model.pkl
a2_RestaurantReviews_FreshDump.tsv     c2_Classifier_Sentiment_Model
b1_Sentiment_Analysis_Model.ipynb      c3_Predicted_Sentiments_Fresh_Dump.tsv
b2_Sentiment_Predictor.ipynb	       titles.tsv


In [2]:
import csv
import requests
from bs4 import BeautifulSoup

# Provide the URL of the webpage
url = 'https://www.flipkart.com/apple-iphone-14-midnight-128-gb/product-reviews/itm9e6293c322a84?pid=MOBGHWFHECFVMDCX&lid=LSTMOBGHWFHECFVMDCXSSCYWA&marketplace=FLIPKART'  # Replace with your actual website link

# Send a GET request to the webpage
i=1
alltitles=[]
newlen=1
prelen=0
while newlen>prelen and newlen<500:
    print(newlen,prelen)
    prelen=newlen
    urll=url+"&page="+str(i)
    i+=1
    response = requests.get(urll)
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the elements containing the desired titles
    title_elements = soup.find_all('p', class_='_2-N8zT')

    # Extract the text from each title element
    titles = [title_element.get_text(strip=True) for title_element in title_elements]
    alltitles.extend(titles)
    newlen=len(alltitles)


      # Define the CSV file path
csv_file = 'titles.tsv'  # Replace with the desired file name or path
# Write the titles to the CSV file
print(alltitles)
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])
    writer.writerows([[title] for title in alltitles])


print('Data saved to', csv_file)


1 0
10 1
20 10
30 20
40 30
50 40
60 50
70 60
80 70
90 80
100 90
110 100
120 110
130 120
140 130
150 140
160 150
170 160
180 170
190 180
200 190
210 200
220 210
230 220
240 230
250 240
260 250
270 260
280 270
290 280
300 290
310 300
320 310
330 320
340 330
350 340
360 350
370 360
380 370
390 380
400 390
410 400
420 410
430 420
440 430
450 440
460 450
470 460
480 470
490 480
['Just wow!', 'Fabulous!', 'Great product', 'Very Good', 'Highly recommended', 'Brilliant', 'Good quality product', 'Just wow!', 'Wonderful', 'Great product', 'Must buy!', 'Terrific purchase', 'Best in the market!', 'Must buy!', 'Best in the market!', 'Awesome', 'Must buy!', 'Excellent', 'Mind-blowing purchase', 'Super!', 'Classy product', 'Waste of money!', 'Classy product', 'Best in the market!', 'Terrific', 'Simply awesome', 'Perfect product!', 'Super!', 'Excellent', 'Brilliant', 'Really Nice', 'Delightful', 'Good choice', 'Excellent', 'Fabulous!', 'Best in the market!', 'Classy product', 'Mind-blowing purchase', 

In [3]:
dataset=pd.read_csv('titles.tsv',delimiter = '\t', quoting = 3)
#dataset = pd.read_csv('a2_RestaurantReviews_FreshDump.tsv', delimiter = '\t', quoting = 3)
dataset.head()
#dataset['Review'][0]="not good"

,Review
0,Just wow!
1,Fabulous!
2,Great product
3,Very Good
4,Highly recommended


###Data cleaning

In [4]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
corpus=[]

for i in range(0, newlen):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

###Data transformation

In [9]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='c1_BoW_Sentiment_Model.pkl'
# cv = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open('./drive/MyDrive/Colab Notebooks/2 Sentiment Analysis (Basic)/3.1 BoW_Sentiment Model.pkl', "rb")))
cv = pickle.load(open(cvFile, "rb"))


In [10]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(500, 1420)

###Predictions (via sentiment classifier)

In [11]:
import joblib
classifier = joblib.load('c2_Classifier_Sentiment_Model')

In [12]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

[1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1
 0 1 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 0 1 0 1 1
 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 0 0 0
 0 1 1 1 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1
 0 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1
 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1
 1 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0
 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 1 0
 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0 0 0
 1 1 1 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0
 1 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0
 0 0 1 1 0 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 0 0 1 0 1 0 1 0 1 0 1 1 1
 1 1 0 0 1 1 0 0 1 1 1 0 

In [13]:
dataset['predicted_label'] = y_pred.tolist()
dataset.head()
l=list(dataset['predicted_label'])
totalpositives=l.count(1);
review=totalpositives/len(l)*100
print(totalpositives,len(l))
#print(type(dataset['predicted_label']))

254 500


In [14]:
dataset.to_csv("c3_Predicted_Sentiments_Fresh_Dump.tsv", sep='\t', encoding='UTF-8', index=False)

In [16]:
dataset=pd.read_csv('c3_Predicted_Sentiments_Fresh_Dump.tsv',delimiter = '\t', quoting = 3)
#dataset = pd.read_csv('a2_RestaurantReviews_FreshDump.tsv', delimiter = '\t', quoting = 3)
dataset.head(200)

,Review,predicted_label
0,Just wow!,1
1,Fabulous!,1
2,Great product,1
3,Very Good,0
4,Highly recommended,0
...,...,...
195,Simply awesome,0
196,Did not meet expectations,0
197,Super!,0
198,Good choice,0


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['predicted_label'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_1, *['predicted_label'], **{})
chart